# Gerenciador de finanças

Bibliotecas e opções

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
from plotly import tools
import plotly.graph_objs as go
from lib import lib as l

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
lcolors = np.array(['#13E881', '#FF5949', '#FFB84C', '#A7BEFA'])
dcolors = np.array(['#268040', '#800B00', '#A13808', '#464CC2'])

## Leitura dos dados

In [3]:
dg = pd.read_csv('br_ganhos.csv')
dg['Data'] = dg['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
dg.tail()

,Nome,Valor,Data,Conta,Area
18,CDI Nu,2.270,2019-11-30,Nu,Investimento
19,CDI Nu,6.160,2019-11-30,Nu,Investimento
20,CDI Nu,0.370,2019-11-30,Nu,Investimento
21,CDI Nu,0.020,2019-11-30,Nu,Investimento
22,ETF,148.740,2019-11-30,Easy,Investimento


In [4]:
df = pd.read_csv('br_gastos.csv')
df['Data'] = df['Data'].apply(lambda x : dt.datetime.strptime(str(x),'%d/%m/%y').date())
df.tail()

,Nome,Valor,Data,Pagamento,Area,Pessoal
52,Agua,72.333,2019-12-31,Pais,Moradia,Sim
53,Luz,138.667,2019-12-31,Pais,Moradia,Sim
54,Internet,191.333,2019-12-31,Pais,Moradia,Sim
55,Catherine,124.900,2019-11-29,Pais,Eletronicos,Sim
56,Impressao,4.500,2019-11-28,Cash,Estudo,Sim


In [5]:
dtr = pd.read_csv('br_transf.csv')
dtr.tail()

,De,Para,Valor
0,BB,Nu,12491.070
1,Nu,Easy,1455.000
2,Nu,Easy,731.400
3,Nu,Easy,870.270


## Saldo atual

In [6]:
l.calc_funds(dg, df, dtr, [855.51, 0, 269, 3187.3, 14585.12])

,Saldo,Lido,Erro
Alelo,927.520,855.510,72.010
BB,0.000,0.000,0.000
Cash,2.500,269.000,-266.500
Easy,3197.000,3187.300,9.700
Nu,14499.150,14585.120,-85.970


## Análise mensal

In [7]:
monthly_earnings = l.window_mean(l.group_sum(dg, group=[]))
monthly_spendings = l.window_mean(l.group_sum(df, group=[]))
monthly_real_spendings = l.window_mean(l.group_sum(df[df['Pagamento'] != 'Pais'], group=[]))

### Balanço

In [8]:
fig = l.plot_bar([monthly_earnings, monthly_spendings, monthly_real_spendings], y='Valor', 
                 color=lcolors[0:3], 
                 name=['Ganho', 'Gasto', 'Gasto Real'])
l.plot_line([monthly_earnings, monthly_spendings, monthly_real_spendings], fig=fig, y='Avg', 
            color=dcolors[0:3], 
            name=['Ganho médio', 'Gasto médio', 'Gasto Real médio'])

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': '#13E881'},
              'na…

### Saldo

In [9]:
fund = l.calc_balance(monthly_earnings, monthly_real_spendings)
fund = l.window_mean(fund, select='Var')
fund

,Ganho,Ganho médio,Gasto,Gasto médio,Saldo,Var,Crescimento,Avg
Ago/19,12508.530,5314.627,0.000,0.000,12508.530,12508.530,0.000,5208.027
Set/19,115.940,5314.627,123.550,182.637,12500.920,-7.610,-0.061,5208.027
Out/19,3319.410,5314.627,196.250,182.637,15624.080,3123.160,24.983,5208.027
Nov/19,3319.180,2251.510,228.110,182.637,18715.150,3091.070,19.784,2068.873
Dez/19,0.000,0.000,88.980,171.113,18626.170,-88.980,-0.475,2041.750


In [10]:
l.plot_stacked_area(fund, y='Saldo', color=lcolors[3])

FigureWidget({
    'data': [{'marker': {'color': '#A7BEFA'},
              'stackgroup': 'one',
              …

In [11]:
fig = l.plot_bar(fund, y='Var', color=lcolors[3], name='Variacao')
l.plot_line(fund, fig=fig, y='Avg', color=dcolors[3], name='Variacao média')

FigureWidget({
    'data': [{'marker': {'color': '#A7BEFA'},
              'name': 'Variacao',
              '…

## Matriz de finanças

In [17]:
area_spendings = l.group_sum(df[df['Pagamento'] != 'Pais'], group=['Area'])
area_spendings_p = l.group_sum(df, group=['Area'])
area_earnings = l.group_sum(dg, group=['Area'])

### Gastos

In [13]:
l.plot_stacked_area(area_spendings, color=lcolors, name=area_spendings.columns)

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': '#13E881'},
              'na…

In [14]:
l.plot_pie(area_spendings.sum(), colors=lcolors)

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['Alimentacao', 'Banco', 'Eletronicos'…

In [18]:
l.plot_pie(area_spendings_p.sum(), colors=lcolors)

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['Alimentacao', 'Banco', 'Eletronicos'…

### Ganhos

In [15]:
l.plot_stacked_area(area_earnings, color=lcolors, name=area_earnings.columns)

FigureWidget({
    'data': [{'hoverinfo': 'y',
              'marker': {'color': '#13E881'},
              'na…

In [22]:
l.plot_pie(area_earnings.sum(), colors=lcolors)

FigureWidget({
    'data': [{'hole': 0.4,
              'labels': array(['Investimento', 'Pais', 'Salario'], d…